In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pycaret.time_series import *
from sktime.forecasting.model_selection import SlidingWindowSplitter
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
warnings.filterwarnings('ignore')

In [2]:
%pip install 'pycaret[full]'

  Using cached shap-0.44.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (24 kB)
  Using cached interpret-0.5.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached umap-learn-0.5.5.tar.gz (90 kB)
  Preparing metadata (setup.py) ... done
  Using cached PyYAML-5.3.1.tar.gz (269 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      /opt/homebrew/lib/python3.11/site-packages/setuptools/config/setupcfg.py:293: _DeprecatedConfig: Deprecated config in `setup.cfg`
      !!
      
              ********************************************************************************
              The license_file parameter is deprecated, use license_files instead.
      
              This deprecation is overdue, please update your project and remove deprecated
              calls to avoid build errors in the future.
      
              See https://setuptools.pypa.io/en/l

In [7]:
%pip install 'packaging==21.3'

  Using cached packaging-21.3-py3-none-any.whl.metadata (15 kB)
Using cached packaging-21.3-py3-none-any.whl (40 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 23.2
    Uninstalling packaging-23.2:
      Successfully uninstalled packaging-23.2
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install --upgrade 'packaging'
%pip install --upgrade 'pycaret'

  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
df = pd.read_csv("train.csv")

df = df[['DATETIMEDATA','PM25']]
df = df.set_index('DATETIMEDATA')
df

,PM25
DATETIMEDATA,
2024-01-01 00:00:00,16.5
2024-01-01 01:00:00,33.6
2024-01-01 02:00:00,34.2
2024-01-01 03:00:00,20.1
2024-01-01 04:00:00,16.5
...,...
2024-02-28 19:00:00,23.6
2024-02-28 20:00:00,25.2
2024-02-28 21:00:00,23.2


In [3]:
from sktime.forecasting.model_selection import SlidingWindowSplitter

In [4]:
import numpy as np
from sktime.forecasting.model_selection import SlidingWindowSplitter
ts = np.arange(10)
splitter = SlidingWindowSplitter(fh=np.arange(1,3), window_length=3, step_length=1)
list(splitter.split(ts))

[(array([0, 1, 2]), array([3, 4])),
 (array([1, 2, 3]), array([4, 5])),
 (array([2, 3, 4]), array([5, 6])),
 (array([3, 4, 5]), array([6, 7])),
 (array([4, 5, 6]), array([7, 8])),
 (array([5, 6, 7]), array([8, 9]))]

In [5]:
df.size

1416

In [12]:
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
from sktime.forecasting.model_selection import SlidingWindowSplitter
exp = TSForecastingExperiment()
exp.setup(data = df, target='PM25' ,coverage=0.90, fold_strategy=SlidingWindowSplitter(fh=np.arange(1,23), window_length=130, step_length=130)) # using the past 6 months data to make prediction for the next month and moving half a year forward. 

,Description,Value
0,session_id,5600
1,Target,PM25
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(1416, 1)"
5,Transformed data shape,"(1416, 1)"
6,Transformed train set shape,"(1394, 1)"
7,Transformed test set shape,"(22, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,SlidingWindowSplitter


In [13]:
exp.plot_model(plot='train_test_split')


In [14]:
exp.compare_models(sort='rmse')


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
theta,Theta Forecaster,0.9651,0.9284,3.8063,4.6661,0.2546,0.2392,-0.6583,0.0090
ets,ETS,0.9788,0.9296,3.8636,4.6685,0.2569,0.2424,-0.6125,0.0300
exp_smooth,Exponential Smoothing,0.9891,0.9412,3.9077,4.7345,0.2601,0.2442,-0.6682,0.0220
auto_arima,Auto ARIMA,1.0270,0.9706,4.1433,4.9826,0.2850,0.2441,-1.0743,4.6950
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,1.0456,1.0206,4.3171,5.3532,0.3077,0.2533,-2.4390,0.1120
croston,Croston,1.1174,1.0393,4.5681,5.4240,0.3231,0.2680,-1.8330,0.0050
grand_means,Grand Means Forecaster,1.1016,1.0475,4.5233,5.4851,0.3237,0.2702,-2.0515,0.0070
naive,Naive Forecaster,1.1863,1.0899,4.7145,5.6018,0.2962,0.2849,-1.2918,0.2390
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,1.1169,1.0631,4.6313,5.6107,0.3427,0.2675,-3.5448,0.1090
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,1.1169,1.0631,4.6313,5.6107,0.3427,0.2675,-3.5448,0.0980


ThetaForecaster(sp=24)

In [15]:
ets = exp.create_model('ets')


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-01-06 09:00,1.3751,1.1114,4.7518,5.4318,0.2793,0.2443,-0.4631
1,2024-01-11 19:00,0.9586,1.1507,2.8985,4.4101,0.1473,0.1616,0.0455
2,2024-01-17 05:00,0.5791,0.5231,3.1185,3.4413,0.2184,0.2055,-0.4769
3,2024-01-22 15:00,0.8293,0.7027,3.8944,4.3823,0.4091,0.3358,-2.4775
4,2024-01-28 01:00,1.0210,1.0621,4.1207,5.1040,0.2158,0.2474,-1.0892
5,2024-02-02 11:00,1.3776,1.2620,5.4871,6.2573,0.3489,0.2789,-0.6073
6,2024-02-07 21:00,0.5641,0.5307,3.5780,4.2738,0.2050,0.1801,0.6363
7,2024-02-13 07:00,0.7436,0.7076,3.1819,4.0588,0.1885,0.1700,-0.5839
8,2024-02-18 17:00,1.2167,1.1831,3.5469,4.3996,0.2443,0.2153,0.1164
9,2024-02-24 03:00,1.1228,1.0630,4.0581,4.9265,0.3122,0.3855,-1.2251


In [16]:
tuned_ets = exp.tune_model(ets)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-01-06 09:00,1.2109,1.0317,4.1843,5.0424,0.2502,0.2194,-0.2608
1,2024-01-11 19:00,1.0036,1.2152,3.0345,4.6571,0.1511,0.1700,-0.0644
2,2024-01-17 05:00,0.9500,0.9389,5.1159,6.1764,0.3845,0.2968,-3.7575
3,2024-01-22 15:00,nan,nan,nan,nan,nan,nan,nan
4,2024-01-28 01:00,0.9655,0.9491,3.8966,4.5609,0.2107,0.2314,-0.6683
5,2024-02-02 11:00,2.8938,2.4443,11.5259,12.1195,0.7025,0.4966,-5.0297
6,2024-02-07 21:00,0.6079,0.5550,3.8552,4.4696,0.2045,0.1885,0.6022
7,2024-02-13 07:00,1.0109,0.9773,4.3256,5.6061,0.2708,0.2178,-2.0218
8,2024-02-18 17:00,nan,nan,nan,nan,nan,nan,nan
9,2024-02-24 03:00,1.0881,0.9887,3.9325,4.5821,0.3243,0.3952,-0.9249


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.8s finished


In [18]:
holdout_pred = exp.predict_model(tuned_ets)


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,ETS,0.7288,0.7853,3.1752,4.4837,0.1555,0.1691,-0.1106


In [19]:
holdout_pred

,y_pred
2024-02-28 02:00,16.3722
2024-02-28 03:00,15.2906
2024-02-28 04:00,15.9119
2024-02-28 05:00,17.6367
2024-02-28 06:00,18.7088
2024-02-28 07:00,19.7007
2024-02-28 08:00,20.7016
2024-02-28 09:00,18.1012
2024-02-28 10:00,16.3520
2024-02-28 11:00,16.1068


In [20]:
final_model = exp.finalize_model(tuned_ets)


In [21]:
exp.plot_model(final_model, plot='forecast', data_kwargs={'fh':24})


In [22]:
unseen_predictions = exp.predict_model(final_model, fh=168)
unseen_predictions

,y_pred
2024-02-29 00:00,21.1983
2024-02-29 01:00,20.9325
2024-02-29 02:00,21.3039
2024-02-29 03:00,19.9072
2024-02-29 04:00,20.8009
2024-02-29 05:00,23.0352
2024-02-29 06:00,24.4347
2024-02-29 07:00,25.7298
2024-02-29 08:00,27.0068
2024-02-29 09:00,23.6565
